In [5]:
import torch
from torchview import draw_graph
from  generator.parser import Parser
import generator.bricks as bricks
import sys
import time
import torch.nn.functional as F

In [17]:
# Тест создания моделей из выражения строки
device = "cuda"
parser = Parser() # создаем парсер
s1 = "output={{@4->relu+@8->relu}^2}%2->@16->softmax->linear(5);"
s2 = "output={{@16->relu+@16->sigmoid}^4}%8->@16;"
s3 = "output={{@64->relu}^64}%64;"
s4 = "output = linear(5) -> softmax;"
s5 = "output = {@5->@20 + @10->@20 + @20} -> softmax;"
start_time = time.time()
# создаем модель из строки s5
modules = parser.from_str(s5)
# # Создаем модули их json-файла
# modules = parser.from_json('nntest.json')
model = modules['output'].to(device)
end_time = time.time()
print(f"Время генерации моделей: {end_time-start_time}")



Parser: инициализация ...
ERROR: В модуле не определен входной блок
Время генерации моделей: 0.008736610412597656


In [18]:
# Тестируем работу модели на тестовом тензоре
x = torch.randn(1, 5).to(device)
output2 = model.to(device)(x)
print(output2)
# Получим один из элементов модели
print(modules['output'].left)
print(modules['output'].left.left.right)
modules

tensor([[0.0269, 0.0407, 0.0728, 0.0269, 0.0492, 0.0428, 0.0797, 0.0830, 0.0366,
         0.0536, 0.0561, 0.0720, 0.0317, 0.0342, 0.0350, 0.0379, 0.0744, 0.0693,
         0.0414, 0.0358]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
{{{linear(5)->linear(20)}+{linear(10)->linear(20)}}+linear(20)}
{linear(10)->linear(20)}


{'output': {{{{linear(5)->linear(20)}+{linear(10)->linear(20)}}+linear(20)}->softmax}}

In [14]:
# Подсчитаем размер модели
params_count = sum(p.numel() for p in modules['output'].parameters())
print(f"Параметров: {params_count}, время создания: ", end_time - start_time)
output2.shape

Параметров: 550, время создания:  0.009173870086669922


torch.Size([1, 20])

In [15]:
# Представление модели в виде выражения
model.expr_str(expand=False)

'{{{{linear(5)->linear(20)}+{linear(10)->linear(20)}}+linear(20)}->softmax}'

In [22]:
# Нарисуем модель
input_size = (1, 5)
pic_path = './pic/'
model_graph = draw_graph(
    model, 
    input_size=input_size,
    graph_name='test',
    graph_dir='LR',
    depth=8,
    hide_inner_tensors=True,
    hide_module_functions=True,
    save_graph=True,
    expand_nested=True,
    show_shapes=True,
    filename='test',
    directory=pic_path
)

# model_graph.resize_graph(scale=3)
# model_graph.visual_graph.view('./pic/test',)

In [26]:
x = torch.randn(input_size).to(device)
chunk = modules['output'].left.right
print(chunk(x))
model_graph = draw_graph(
    chunk, 
    input_size=input_size,
    graph_name='left.right',
    graph_dir='LR',
    depth=8,
    hide_inner_tensors=True,
    hide_module_functions=True,
    save_graph=True,
    expand_nested=True,
    show_shapes=True,
    filename='left.right',
    directory='./pic/'
)
# Разделим на две части, соединенные параллельно
left, right = chunk.decompose()
new_chunk = bricks.Connector(left, right).to("cuda")
print(new_chunk(x))
model_graph = draw_graph(
    new_chunk, 
    input_size=input_size,
    graph_name='decompose',
    graph_dir='LR',
    depth=8,
    hide_inner_tensors=True,
    hide_module_functions=True,
    save_graph=True,
    expand_nested=True,
    show_shapes=True,
    filename='decompose',
    directory='./pic/'
)

tensor([[-0.1565,  0.3500, -0.3436,  0.3257,  0.0636,  0.4658,  0.6467,  0.0350,
         -0.0691, -0.6981,  1.4110, -0.1775,  0.6306,  0.1815,  0.0094, -0.4796,
          0.3549, -0.6143,  0.0438, -0.2355]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([[ 0.9092, -0.6238,  0.1989,  0.1603, -1.8952,  0.2508,  0.0374, -0.5332,
          0.0413,  0.5759,  0.3830,  0.1071, -0.4529,  0.4732, -0.6295,  0.4190,
         -0.6940,  0.9568,  0.5844, -0.3995, -0.5055, -0.3539,  0.0489,  0.9234,
          0.3009]], device='cuda:0', grad_fn=<CatBackward0>)
